# Lab 5: Port Detection using AIS Data

This notebook implements a port detection algorithm using Automatic Identification System (AIS) vessel data. The approach involves:
1. Parallel data loading and filtering
2. Speed calculation and low-speed point identification
3. Batch grouping of low-speed areas
4. Clustering to identify ports
5. Port visualization with statistics

# Data

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import folium
from sklearn.cluster import DBSCAN
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# Data file path
file_path = '../../lab1/data/aisdk-2025-02-09.csv'
print(f"Loading data from: {file_path}")

Loading data from: ../../lab1/data/aisdk-2025-02-09.csv


## Filter out the noise and prepare data for port detection. (Think what is the noise in this solution)

- Mobile units only
- Remove the sudden jump points
- for each MMSI
    - Sort by time
    - Calculate the distance, time difference and speeed between points
    - Remove all points with speed more than 5 kmh
    - Remove all points that have 0 time difference, but a distance of more than 10 metres
    - Combine the points into batches - traverse the filtered points, and group them together if they are nearby in terms of space and time.

In [2]:
# Constants
EARTH_RADIUS = 6378  # Earth's radius in kilometers
SUDDEN_JUMP_THRESHOLD = 0.01  # Distance threshold for sudden jumps in km
LOW_SPEED_THRESHOLD = 5  # Low speed threshold for port detection in km/h
MINIMUM_POINTS_FOR_BATCH = 10

def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on Earth."""
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return EARTH_RADIUS * c

def include_spacetime_features(vessel_data):
    # Calculate speeds using vectorized operations
    prev_lats = vessel_data['Latitude'].shift(1)
    prev_lons = vessel_data['Longitude'].shift(1)
    prev_times = vessel_data['Timestamp'].shift(1)
    
    # Calculate time differences in hours
    time_diffs = (vessel_data['Timestamp'] - prev_times).dt.total_seconds() / 3600
    
    # Calculate distances (vectorized)
    distances = np.array([
        calculate_distance(prev_lat, prev_lon, lat, lon) 
        if pd.notna(prev_lat) else 0
        for prev_lat, prev_lon, lat, lon in zip(prev_lats, prev_lons, vessel_data['Latitude'], vessel_data['Longitude'])
    ])

    return distances, time_diffs

def process_vessel_data(vessel_group):
    """Process a single vessel's data to calculate speeds and filter low-speed points."""
    mmsi = vessel_group.iloc[0]['MMSI']
    
    # Sort by timestamp and remove duplicates
    vessel_data = vessel_group.sort_values('Timestamp').drop_duplicates('Timestamp').reset_index(drop=True)
    
    if len(vessel_data) < 2:
        return pd.DataFrame()  # Not enough data points
    
    # Calculate distances and time differences
    distances, time_diffs = include_spacetime_features(vessel_data)
    
    # Identify sudden jumps (zero time diff with significant distance change)
    sudden_jumps = (time_diffs == 0) & (distances > SUDDEN_JUMP_THRESHOLD)
    
    # Calculate speeds (avoiding division by zero)
    speeds = np.divide(distances, time_diffs, out=np.zeros_like(distances), where=time_diffs > 0)
    vessel_data['speed_kmh'] = speeds
    
    # Remove sudden jumps and keep only low-speed points (potential port areas)
    valid_points = (~sudden_jumps) & ((vessel_data['speed_kmh'] <= LOW_SPEED_THRESHOLD) | (vessel_data.index == 0))
    low_speed_data = vessel_data[valid_points].copy()

    if len(low_speed_data) > 0:
        return low_speed_data[['MMSI', 'Timestamp', 'Latitude', 'Longitude', 'speed_kmh', 'Type of mobile']]
    else:
        return pd.DataFrame()

def create_vessel_batches(vessel_data, time_window_hours=1/60, spatial_threshold_km=0.1):
    """Create batches for a single vessel from its low-speed data."""
    if len(vessel_data) < MINIMUM_POINTS_FOR_BATCH:
        return []
    
    # Sort by timestamp
    distances, time_diffs = include_spacetime_features(vessel_data)
    vessel_data["distance"] = distances
    vessel_data["time_diff"] = time_diffs
    mmsi = vessel_data.iloc[0]['MMSI']
    
    batches = []
    current_batch = []
    
    for idx, row in vessel_data.iterrows():
        if not current_batch:
            current_batch.append(row)
        else:
            # Check if current point should be added to current batch
            time_diff = row['time_diff']
            distance = row['distance']
            if time_diff <= time_window_hours and distance <= spatial_threshold_km:
                current_batch.append(row)
            else:
                # Save current batch if it has enough points
                if len(current_batch) >= MINIMUM_POINTS_FOR_BATCH:
                    batch_df = pd.DataFrame(current_batch)
                    center_lat = batch_df['Latitude'].mean()
                    center_lon = batch_df['Longitude'].mean()
                    start_time = batch_df['Timestamp'].min()
                    end_time = batch_df['Timestamp'].max()
                    
                    batches.append({
                        'mmsi': mmsi,
                        'center_lat': center_lat,
                        'center_lon': center_lon,
                        'start_time': start_time,
                        'end_time': end_time,
                        'point_count': len(current_batch),
                        'mobile_type': current_batch[0]['Type of mobile']
                    })
                
                # Start new batch
                current_batch = [row]
    
    # Don't forget the last batch
    if len(current_batch) >= MINIMUM_POINTS_FOR_BATCH:
        batch_df = pd.DataFrame(current_batch)
        center_lat = batch_df['Latitude'].mean()
        center_lon = batch_df['Longitude'].mean()
        start_time = batch_df['Timestamp'].min()
        end_time = batch_df['Timestamp'].max()
        
        batches.append({
            'mmsi': mmsi,
            'center_lat': center_lat,
            'center_lon': center_lon,
            'start_time': start_time,
            'end_time': end_time,
            'point_count': len(current_batch),
            'mobile_type': current_batch[0]['Type of mobile']
        })
    
    return batches

def load_filter_and_process_chunk(chunk):
    """Load, filter a chunk of data, process vessel data, and create batches"""
    # Parse timestamp
    chunk['Timestamp'] = pd.to_datetime(chunk['# Timestamp'])
    
    # Filter out Base Stations and N/A mobile units
    chunk = chunk[chunk['Type of mobile'] != 'Base Station'].copy()
    chunk = chunk.dropna(subset=['Type of mobile', 'MMSI', 'Latitude', 'Longitude'])
    
    # Remove invalid coordinates
    chunk = chunk[
        (chunk['Latitude'].between(-90, 90)) & 
        (chunk['Longitude'].between(-180, 180))
    ]
    
    if chunk.empty:
        return pd.DataFrame()
    
    # Process each vessel in this chunk and create batches
    all_batches = []
    for mmsi, vessel_group in chunk.groupby('MMSI'):
        # First get low-speed points for this vessel
        vessel_low_speed = process_vessel_data(vessel_group)
        
        if not vessel_low_speed.empty:
            # Then create batches from the low-speed points
            vessel_batches = create_vessel_batches(vessel_low_speed)
            all_batches.extend(vessel_batches)
    
    if all_batches:
        return pd.DataFrame(all_batches)
    else:
        return pd.DataFrame()

In [3]:
# Load, process data, and create batches in chunks with parallel processing

chunk_size =   300000
# chunk_size = 1000000 # optimal for lab2

file_size = 15366011 # Size of the file lines
tbar_total = file_size // chunk_size + 1
tbar = tqdm(total=tbar_total, desc="Processing chunks", unit="chunk")

# Create a generator of chunks
chunks = pd.read_csv(file_path, chunksize=chunk_size)
all_results = []

# Read chunks and process them in parallel
with ProcessPoolExecutor(max_workers=8) as executor:
    
    future_to_chunk = {
        executor.submit(load_filter_and_process_chunk, chunk): i for i, chunk in enumerate(chunks)
    }
    for future in as_completed(future_to_chunk):
        chunk_index = future_to_chunk[future]
        tbar.update(1)
        try:
            chunk_results = future.result()
            all_results.append(chunk_results)
            # print(f"Processed chunk {chunk_index+1}")
        except Exception as e:
            print(f"Error processing chunk {chunk_index+1}: {e}")

Processing chunks: 100%|██████████| 52/52 [02:24<00:00,  1.62s/chunk]

In [4]:
# Concat all batch results into a single DataFrame
batches_df = pd.concat(all_results, ignore_index=True)
print(f"Created {len(batches_df)} batches from {batches_df['mmsi'].nunique()} vessels")

print(f"Average batch size: {batches_df['point_count'].mean():.1f} points")
print(f"Batch duration range: {(batches_df['end_time'] - batches_df['start_time']).dt.total_seconds().min()/3600:.1f} - {(batches_df['end_time'] - batches_df['start_time']).dt.total_seconds().max()/3600:.1f} hours")


print(f"Unique vessels: {batches_df['mmsi'].nunique()}")
print(f"Mobile types in batches: {batches_df['mobile_type'].value_counts().head()}")
print(f"Total points in all batches: {batches_df['point_count'].sum()}")

Created 34196 batches from 1075 vessels
Average batch size: 123.3 points
Batch duration range: 0.0 - 0.5 hours
Unique vessels: 1075
Mobile types in batches: mobile_type
Class A    33261
Class B      875
AtoN          60
Name: count, dtype: int64
Total points in all batches: 4217775


## Create an algorithm for port detection.  

- From the code above, we have a lot of batches. Each batch represents a vessel that has moved slowly in on place for a while. There might be multiple batches for the same vessel.
- First, cluster the batches for each MMSI seperately - this way we save compute
- If the vessel has been to the same location (allegedly port) multiple times, we will only keep one copy.
- Then we do the clustering for all the ships, obtaining ports
- Remove any ports with less than 10 unique MMSI's

In [5]:
def cluster_batches(batches_df, min_samples, cluster_radius_km=1.0):
    """Cluster batches using DBSCAN"""
    if len(batches_df) < 2:
        print("Not enough batches for clustering")
        return []
    
    n = len(batches_df)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            dist = calculate_distance(
                batches_df.iloc[i]['center_lat'], 
                batches_df.iloc[i]['center_lon'], 
                batches_df.iloc[j]['center_lat'], 
                batches_df.iloc[j]['center_lon']
            )
            distance_matrix[i, j] = distance_matrix[j, i] = dist

    clustering = DBSCAN(eps=cluster_radius_km, min_samples=min_samples, metric="precomputed").fit(
        distance_matrix
    )
    return clustering.labels_.tolist()

# Similar function to above, but using a more approximate method for clustering
def cluster_batches_approx(batches_df, min_samples, cluster_radius_km=1.0):
    """Cluster batches using DBSCAN"""
    if len(batches_df) < 2:
        print("Not enough batches for clustering")
        return pd.DataFrame()
    
    # Prepare coordinates for clustering
    coordinates = batches_df[['center_lat', 'center_lon']].values
    
    # Convert radius from km to degrees (approximate)
    # 1 degree ≈ 111 km at equator
    eps_degrees = cluster_radius_km / 111.0
    
    # Perform DBSCAN clustering
    clustering = DBSCAN(eps=eps_degrees, min_samples=min_samples).fit(coordinates)

    return clustering.labels_.tolist()

In [6]:
def concat_vessel_batches(group):
    """
    For a single vessel, join all batches that were closed together into one.
    Return the average center_lat, center_lon, min start_time, max end_time, and sum of point_count.
    """
    return pd.Series({
        'mmsi': group['mmsi'].iloc[0],
        'center_lat': group['center_lat'].mean(),
        'center_lon': group['center_lon'].mean(),
        'start_time': group['start_time'].min(),
        'end_time': group['end_time'].max(),
        'point_count': group['point_count'].sum(),
        'mobile_type': group['mobile_type'].mode()[0]  # Most common type
    })

def process_batches(chunk):
    group = chunk[1]
    if len(group) < 2:
        return group
    
    # Cluster the batches for this vessel
    labels = cluster_batches_approx(group, min_samples=1)
    group['cluster'] = labels
    group = group.groupby('cluster').apply(concat_vessel_batches).reset_index(drop=True)
    return group

In [ ]:
chunks = batches_df.groupby('mmsi')
all_results = []

t_bar = tqdm(total=len(chunks), desc="Processing batches")

# Read chunks and process them in parallel
with ProcessPoolExecutor(max_workers=8) as executor:
    
    future_to_chunk = {
        executor.submit(process_batches, chunk): i for i, chunk in enumerate(chunks)
    }
    for future in as_completed(future_to_chunk):
        t_bar.update(1)
        chunk_index = future_to_chunk[future]
        try:
            chunk_results = future.result()
            all_results.append(chunk_results)
            # print(f"Processed chunk {chunk_index+1}")
        except Exception as e:
            print(f"Error processing chunk {chunk_index+1}: {e}")

In [8]:
filtered_batches_df = pd.concat(all_results, ignore_index=True)
print(f"Filtered batches: {len(filtered_batches_df)} from {len(batches_df)} original batches")
# unique MMSI 
print(f"Unique vessels after filtering: {filtered_batches_df['mmsi'].nunique()}")
# before filtering
print(f"Unique vessels before filtering: {batches_df['mmsi'].nunique()}")

Filtered batches: 4143 from 34196 original batches
Unique vessels after filtering: 1075
Unique vessels before filtering: 1075


In [9]:
# Cluster the filtered batches into ports
port_clusters = cluster_batches_approx(filtered_batches_df, min_samples=10, cluster_radius_km=1.0)
filtered_batches_df['cluster'] = port_clusters

In [10]:
# set the cluster to -1 if it does not have more than 10 unique MMSI
small_cluster = filtered_batches_df.groupby('cluster').apply(lambda x: x['mmsi'].nunique() < 10)
filtered_batches_df.loc[filtered_batches_df['cluster'].isin(small_cluster[small_cluster].index), 'cluster'] = -1

In [11]:
valid_ports = filtered_batches_df[filtered_batches_df['cluster'] != -1]
print(f"\nTotal valid port batches: {len(valid_ports)}")
print(f"Vessels in ports: {valid_ports['mmsi'].nunique()}")
print(f"Ports with more than 10 vessels: {valid_ports['cluster'].nunique()}")


Total valid port batches: 487
Vessels in ports: 406
Ports with more than 10 vessels: 26


In [12]:
clustered_batches = valid_ports.reset_index(drop=True)

## Evaluate the relative size of the port.

In [13]:
def calculate_port_stats(clustered_batches):
    """Calculate statistics for each detected port."""
    port_stats = []
    
    for cluster_id in clustered_batches['cluster'].unique():
        
        cluster_data = clustered_batches[clustered_batches['cluster'] == cluster_id]
        
        # Calculate bounding box
        min_lat = cluster_data['center_lat'].min()
        max_lat = cluster_data['center_lat'].max()
        min_lon = cluster_data['center_lon'].min()
        max_lon = cluster_data['center_lon'].max()
        
        # Calculate grid size in km
        lat_size_km = calculate_distance(min_lat, min_lon, max_lat, min_lon)
        lon_size_km = calculate_distance(min_lat, min_lon, min_lat, max_lon)
        area_km2 = lat_size_km * lon_size_km
        
        # Calculate center point
        center_lat = cluster_data['center_lat'].mean()
        center_lon = cluster_data['center_lon'].mean()
        
        # Other statistics
        total_batches = len(cluster_data)
        unique_mmsi = cluster_data['mmsi'].nunique()
        total_points = cluster_data['point_count'].sum()
        mobile_types = cluster_data['mobile_type'].value_counts().to_dict()
        
        port_stats.append({
            'cluster_id': cluster_id,
            'min_lat': min_lat,
            'max_lat': max_lat,
            'min_lon': min_lon,
            'max_lon': max_lon,
            'lat_size_km': lat_size_km,
            'lon_size_km': lon_size_km,
            'area_km2': area_km2,
            'total_batches': total_batches,
            'unique_mmsi': unique_mmsi,
            'total_points': total_points,
            'mobile_types': mobile_types
        })
    
    return pd.DataFrame(port_stats)

port_stats_df = calculate_port_stats(clustered_batches)

print(f"\n=== PORT STATISTICS ===")
for _, port in port_stats_df.iterrows():
    print(f"\nPort {port['cluster_id']}:")
    print(f"  Location: ({port['min_lat']:.4f}, {port['min_lon']:.4f})")
    print(f"  Size: {port['lat_size_km']:.2f} km × {port['lon_size_km']:.2f} km (Area: {port['area_km2']:.2f} km²)")
    print(f"  Vessels: {port['unique_mmsi']} unique MMSI")
    print(f"  Batches: {port['total_batches']}")
    print(f"  Total points: {port['total_points']}")
    # Show top 3 mobile types
    top_types = dict(list(port['mobile_types'].items())[:3])
    print(f"  Top mobile types: {top_types}")


=== PORT STATISTICS ===

Port 0:
  Location: (55.4662, 8.4098)
  Size: 1.69 km × 1.38 km (Area: 2.34 km²)
  Vessels: 27 unique MMSI
  Batches: 27
  Total points: 139413
  Top mobile types: {'Class A': 27}

Port 1:
  Location: (54.6360, 11.3739)
  Size: 0.70 km × 0.81 km (Area: 0.56 km²)
  Vessels: 12 unique MMSI
  Batches: 12
  Total points: 42070
  Top mobile types: {'Class A': 12}

Port 2:
  Location: (54.2580, 9.6214)
  Size: 0.91 km × 0.36 km (Area: 0.33 km²)
  Vessels: 11 unique MMSI
  Batches: 11
  Total points: 827
  Top mobile types: {'Class A': 11}

Port 3:
  Location: (55.3630, 13.1470)
  Size: 0.83 km × 0.94 km (Area: 0.78 km²)
  Vessels: 10 unique MMSI
  Batches: 12
  Total points: 2099
  Top mobile types: {'Class A': 12}

Port 4:
  Location: (54.6525, 11.3478)
  Size: 0.50 km × 0.17 km (Area: 0.09 km²)
  Vessels: 14 unique MMSI
  Batches: 14
  Total points: 13668
  Top mobile types: {'Class A': 14}

Port 5:
  Location: (55.0873, 14.6776)
  Size: 1.92 km × 1.19 km (Area: 2

In [14]:
port_stat = port_stats_df[["cluster_id", "area_km2", "unique_mmsi",  "total_batches","total_points"]].sort_values(by="area_km2", ascending=False)

pd.concat([port_stat.head(3), port_stat.tail(3)], ignore_index=True).rename(columns={
    "cluster_id": "Port ID",
    "area_km2": "Area (km²)",
    "total_batches": "Total Batches",
    "unique_mmsi": "Unique Vessels",
    "total_points": "Data Points"
})

,Port ID,Area (km²),Unique Vessels,Total Batches,Data Points
0,23,4.484977,16,25,1176
1,18,2.395893,51,55,254426
2,0,2.341357,27,27,139413
3,4,0.086667,14,14,13668
4,13,0.045768,18,18,108488
5,17,0.035793,15,15,96632


In [15]:
# Display summary
print(f"\n=== PORT DETECTION SUMMARY ===")
print(f"Total ports detected: {len(port_stats_df)}")
print(f"Total vessels involved: {clustered_batches[clustered_batches['cluster'] != -1]['mmsi'].nunique()}")
print(f"Total low-speed batches in ports: {len(clustered_batches[clustered_batches['cluster'] != -1])}")

# Show largest ports
largest_port = port_stats_df.loc[port_stats_df['area_km2'].idxmax()]
busiest_port = port_stats_df.loc[port_stats_df['unique_mmsi'].idxmax()]

print(f"\nLargest port: Port {largest_port['cluster_id']} ({largest_port['area_km2']:.2f} km²)")
print(f"Busiest port: Port {busiest_port['cluster_id']} ({busiest_port['unique_mmsi']} vessels)")


=== PORT DETECTION SUMMARY ===
Total ports detected: 26
Total vessels involved: 406
Total low-speed batches in ports: 487

Largest port: Port 23 (4.48 km²)
Busiest port: Port 18 (51 vessels)


In [16]:
# Notable ports:
# 23 - largest
# 18 - most vessels
# 17 - smallest

## Visualize ports. (Get creative with it.) It's good to know the location of the port, its relative size, etc.

- Use folium to place a mesh on clusters. Include their size, number of points and unique MMSI count.
- Also mark each batch in the cluster, we can see that they sometimes represent the ship docking locations.

In [17]:
def create_port_map(port_stats_df, clustered_batches):
    """Create an interactive map showing detected ports."""
    if port_stats_df.empty:
        print("No ports to visualize")
        return None
    
    # Calculate map starting pos, start the the first port
    center_lat = port_stats_df['min_lat'].iloc[0]
    center_lon = port_stats_df['min_lon'].iloc[0]
    
    # Create map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=8)
    
    # Color palette for different ports
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 
             'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 
             'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
    
    # Add port areas
    for idx, port in port_stats_df.iterrows():
        color = colors[idx % len(colors)]
        
        # Create bounding box
        bounds = [
            [port['min_lat'], port['min_lon']],
            [port['min_lat'], port['max_lon']],
            [port['max_lat'], port['max_lon']],
            [port['max_lat'], port['min_lon']]
        ]
        if port['cluster_id'] == 17:
            color = "red"
        # Add port boundary
        folium.Polygon(
            locations=bounds,
            color=color,
            weight=3,
            fill=True,
            fillOpacity=0.2,
            popup=folium.Popup(
                f"""<b>Port {port['cluster_id']}</b><br/>
                Size: {port['lat_size_km']:.2f} × {port['lon_size_km']:.2f} km<br/>
                Area: {port['area_km2']:.2f} km²<br/>
                Vessels: {port['unique_mmsi']}<br/>
                Batches: {port['total_batches']}<br/>
                Points: {port['total_points']}""",
                max_width=300
            )
        ).add_to(m)
        
        # Add individual batch points
        port_batches = clustered_batches[clustered_batches['cluster'] == port['cluster_id']]
        for _, batch in port_batches.iterrows():
            folium.CircleMarker(
                location=[batch['center_lat'], batch['center_lon']],
                radius=10,
                popup=f"MMSI: {batch['mmsi']}<br/>Points: {batch['point_count']}<br>Port {port['cluster_id']}",
                color=color,
                fill=True,
                fillOpacity=0.6,
                weight=1
            ).add_to(m)
    
    return m

In [18]:
port_map = create_port_map(port_stats_df, clustered_batches)

# Save the map
port_map.save('detected_ports.html')
print("Map saved as 'detected_ports.html'")

Map saved as 'detected_ports.html'


In [19]:
port_map